# 📘 **00 — Baseline Notebook**

**Notebook Purpose:**
Establish a *minimal, working baseline* model for the Kaggle **Detect Reversal Points in U.S. Equities** competition. This notebook focuses on rapid setup → quick preprocessing → first submission. No feature engineering, no tuning, just a clean, reproducible starting point.

---

**Competition:** *Detect Reversal Points in US Equities*
**Deadline:** December 31, 2025
**Repository:** `Kaggle-Detect-Reversal-Points-in-US-Equities`

---

**Notebook Date Created:** 2025-11-26
**Notebook Last Updated:** 2025-11-26

---

## 🧭 **Goals of This Notebook**

- Load Kaggle training and test data
- Perform *very light* preprocessing appropriate for a baseline
- Train 1–2 simple models (LogReg, LightGBM baseline)
- Generate a valid `submission.csv`
- Store artifacts in `/models/` and `/submissions/`
- Document the baseline performance

---

## 📂 **References**

- Project Plan: `docs/00_overview/reversal_points_project_plan.md`
- Folder Explanations: `docs/01_architecture/02_folder_explanations.md`
- Project Structure: `docs/01_architecture/01_project_structure.md`


In [3]:
import os
import duckdb
import pandas
import numpy
import matplotlib.pyplot as plt
import seaborn as sns

os.getcwd()


'/home/bnelson_regex/projects/machine_learning_projects/kaggle/detect_reversal_points_us_equities/notebooks'

In [8]:
# load dataframe with duck db

df = duckdb.sql("""
    SELECT *
    FROM read_csv_auto(
        '../data/raw/competition_data/train.csv',
        max_line_size=5000000
    )
    LIMIT 5
""").df()

df.head()

,train_id,ticker_id,t,cross_threshold_from_above_100.0,cross_threshold_from_above_100.5,cross_threshold_from_above_101.0,cross_threshold_from_above_101.5,cross_threshold_from_above_102.0,cross_threshold_from_above_102.5,cross_threshold_from_above_103.0,...,zone_102.0,zone_102.5,zone_103.0,zone_97.0,zone_97.5,zone_98.0,zone_98.5,zone_99.0,zone_99.5,class_label
0,0,002,2024-06-10,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,True,False,None
1,1,003,2024-09-18,False,False,False,False,False,False,False,...,False,False,False,False,True,True,False,False,False,None
2,2,006,2023-05-10,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,True,False,None
3,3,003,2024-11-18,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,None
4,4,002,2024-08-21,False,False,False,False,False,False,False,...,True,True,False,False,False,False,False,False,False,None
